In [ ]:
import numpy as np
import random
import secrets
import io

N = 1000

### Генератор Лемера

In [ ]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = np.array([seq % (2**8)], dtype=np.uint8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = np.array([seq >> 24], dtype=np.uint8)

        return seq


In [ ]:
de1 = Linear_Low()
de2 = Linear_High()

print(de1.generate_bytes(N))
print(de2.generate_bytes(N))

### Генератор L20

In [ ]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(20)])
print(smp)
de_L20 = L20(smp)

print(de_L20.generate_bits(N))

### Генератор L89

In [ ]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

print(de_L89.generate_bits(N))

### Генератор Вольфрама

In [ ]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations):
    return (n >> rotations | n << (32-rotations))

def lcs(n: np.uint32, rotations):
    return (n << rotations | n >> (32-rotations))

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))  

        return seq
            

In [ ]:
de_wolfram = Wolfram(1)

print(de_wolfram.generate_bits(N))

### Генератор "Бібліотекар"

In [ ]:
class Librarian:
    def __init__(self, filename):
        file = io.open(filename, mode='r', encoding='utf-8')
        self.text = file.read()

    
    def generate_bytes(self, n: int):
        if len(self.text) < n:
            raise RuntimeError("Nema sliv, odni emotions")

        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = ord(self.text[i])

        return seq
            

In [ ]:
de_Lb = Librarian("fanfiction.txt")

print(de_Lb.generate_bytes(N))

### Генератор BM

In [ ]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array(seq < (self.p - 1) / 2, dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array((seq * 256) // (self.p - 1), dtype=np.uint8) 

        return seq
            

In [ ]:
p = int("CEA42B987C44FA642D80AD9F51F10457690DEF10C83D0BC1BCEE12FC3B6093E3", 16)
a = int("5B88C41246790891C095E2878880342E88C79974303BD0400B090FE38A688356", 16)

de_BM = BM(p, a)

print(de_BM.generate_bits(N))
print(de_BM.generate_bytes(N))

### Генератор Джиффі

In [ ]:
class Geffe:
    def __init__(self, x_init: np.array, y_init: np.array, s_init: np.array):
        self.x = x_init # 11 bits, x11 = x0 + x2
        self.y = y_init # 9 bits,  y9 = y0 + y1 + y3 + y4
        self.s = s_init # 10 bits, s10 = s0 + s3

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = (self.s[0] * self.x[0]) ^ ((1 ^ self.s[0]) * self.y[0])
            # Linear Shift
            self.x[0] = self.x[0] ^ self.x[2]
            self.x = np.roll(self.x, -1)
            self.y[0] = self.y[0] ^ self.y[1] ^ self.y[3] ^ self.y[4]
            self.y = np.roll(self.y, -1)
            self.s[0] = self.s[0] ^ self.s[3]
            self.s = np.roll(self.s, -1)

        return seq
            

In [ ]:
x = np.array([random.randint(0, 1) for _ in range(11)])
y = np.array([random.randint(0, 1) for _ in range(9)])
s = np.array([random.randint(0, 1) for _ in range(10)])

print(x)
print(y)
print(s)

Ge_generator = Geffe(x, y, s)

print(Ge_generator.generate_bits(N))

### Генератор BBS

In [ ]:
class BBS:
    def __init__(self, p, q):
        self.n = p*q

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array([seq % 2], dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(2, self.n - 1) 

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        seq = np.array([seq % (2**8)], dtype=np.uint8) 

        return seq
            

In [ ]:
p = int("D5BBB96D30086EC484EBA3D7F9CAEB07", 16)
q = int("425D2B9BFDB25B9CF6C416CC6E37B59C1F", 16)

de_BBS = BBS(p, q)

print(de_BBS.generate_bytes(N))

### Вбудовані генератори

#### Криптографічно нестійкий генератор

In [ ]:
unsafe_bits_seq = np.array([random.randint(0, 1) for _ in range(N)], dtype=np.uint8)
print(unsafe_bits_seq)

unsafe_bytes_seq = np.array(list(random.randbytes(N)), dtype=np.uint8)
print(unsafe_bytes_seq)

#### Криптографічно стійкий генератор

In [ ]:
safe_bits_seq = np.array([secrets.randbelow(2) for _ in range(N)], dtype=np.uint8)
print(safe_bits_seq)

safe_bytes_seq = np.array(list(secrets.token_bytes(N)), dtype=np.uint8)
print(safe_bytes_seq)